# Notebook (sans Pipeline) — CV + Hyperparamètres

## 0) Imports + dataset

In [ ]:
import numpy as np

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression



X, y = load_wine(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train:", X_train.shape, "Test:", X_test.shape)

## 1) Définir la cross-validation

In [ ]:
# Validation croisée en 5 folds, avec conservation de la proportion des classes
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

Sans pipeline, si tu fais :  
scaler.fit_transform(X_train) puis GridSearchCV(...)


Tu fais un preprocessing “global” avant la CV ⇒ **fuite de données** (data leakage).  
La version correcte consiste à **refaire le scaling dans chaque fold**.  
  
Donc on va faire un mini “GridSearchCV maison” : même logique, mais on maîtrise le preprocessing.

## 2) GridSearch “maison” (sans pipeline) — KNN : chercher le meilleur k

### 2.1 Fonction : score CV pour une valeur de k (scaling refit à chaque fold)

In [ ]:
from sklearn.metrics import accuracy_score

def cv_accuracy_knn_no_pipeline(X, y, k, cv):
    fold_scores = []

    for train_idx, val_idx in cv.split(X, y):
        X_tr, X_va = X[train_idx], X[val_idx]
        y_tr, y_va = y[train_idx], y[val_idx]

        # Scaling refit sur le train du fold uniquement (pas de fuite)
        scaler = StandardScaler()
        X_tr_s = scaler.fit_transform(X_tr)
        X_va_s = scaler.transform(X_va)

        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(X_tr_s, y_tr)

        y_pred = model.predict(X_va_s)
        fold_scores.append(accuracy_score(y_va, y_pred))

    return float(np.mean(fold_scores))

### 2.2 Recherche du meilleur k

In [ ]:
k_values = range(1, 31)
best_k, best_cv = None, -1

for k in k_values:
    score = cv_accuracy_knn_no_pipeline(X_train, y_train, k, cv)
    if score > best_cv:
        best_cv = score
        best_k = k

print("Meilleur k:", best_k)
print("Meilleur score CV:", round(best_cv, 3))

### 2.3 Évaluation finale sur le test (une seule fois)

In [ ]:
# Fit scaler sur tout le train, puis test
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

final_knn = KNeighborsClassifier(n_neighbors=best_k)
final_knn.fit(X_train_s, y_train)

print("Score test final:", round(final_knn.score(X_test_s, y_test), 3))

## 3) Même chose — Logistic Regression : chercher le meilleur C

### 3.1 Fonction score CV (scaling refit à chaque fold)

In [ ]:
def cv_accuracy_logreg_no_pipeline(X, y, C, cv):
    fold_scores = []

    for train_idx, val_idx in cv.split(X, y):
        X_tr, X_va = X[train_idx], X[val_idx]
        y_tr, y_va = y[train_idx], y[val_idx]

        scaler = StandardScaler()
        X_tr_s = scaler.fit_transform(X_tr)
        X_va_s = scaler.transform(X_va)

        model = LogisticRegression(C=C, max_iter=5000)
        model.fit(X_tr_s, y_tr)

        y_pred = model.predict(X_va_s)
        fold_scores.append(accuracy_score(y_va, y_pred))

    return float(np.mean(fold_scores))

### 3.2 Recherche du meilleur C

In [ ]:
C_values = [0.01, 0.1, 1, 10, 100]
best_C, best_cv = None, -1

for C in C_values:
    score = cv_accuracy_logreg_no_pipeline(X_train, y_train, C, cv)
    print(f"C={C:<6} | CV acc={score:.3f}")
    if score > best_cv:
        best_cv = score
        best_C = C

print("Meilleur C:", best_C)
print("Meilleur score CV:", round(best_cv, 3))

### 3.3 Test final

In [ ]:
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

final_lr = LogisticRegression(C=best_C, max_iter=5000)
final_lr.fit(X_train_s, y_train)

print("Score test final:", round(final_lr.score(X_test_s, y_test), 3))

## 4) Mini récap : “Sans pipeline vs avec pipeline”

* ### Sans pipeline :  
  ✅ on peut faire propre, **mais** il faut gérer à la main :  

  * le scaler **dans chaque fold**

  * la répétition de code
 
  * le risque d’erreurs / fuites

* ### Avec pipeline :  
  ✅ même logique, mais :  

  * preprocessing automatiquement dans chaque fold
 
  * `GridSearchCV` devient clean et compact
 
  * c’est la norme en entreprise